In [1]:
// Define GCS paths
val bucketName = "movielens_dataset"
val ratingsPath = s"gs://$bucketName/movie.csv"

bucketName = movielens_dataset
ratingsPath = gs://movielens_dataset/movie.csv


gs://movielens_dataset/movie.csv

In [2]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
  .appName("Movielens movie metadata")
  .getOrCreate()

val moviesDF = spark.read
  .format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load(ratingsPath)

spark = org.apache.spark.sql.SparkSession@26df0048
moviesDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [4]:
println(s"Ratings: ${moviesDF.count()}")

Ratings: 27278


In [5]:
moviesDF.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [20]:
val extractReleaseYear = (title: String) => {
  val pattern = """\((\d{4})\)$"""
  title match {
    case t if t.matches(".*\\(\\d{4}\\)$") =>
      t.replaceAll(".*\\((\\d{4})\\)$", "$1").toInt // Extract year
    case _ =>
      Random.nextInt(51) + 1950 // Random year if not found
  }
}

// Step 2: Convert the DataFrame to an RDD and create JSON objects
val movieJSONRDD = moviesDF.rdd.map(row => {
  val movieId = row.getAs[Int]("movieId")
  val title = row.getAs[String]("title")
  val releaseYear = extractReleaseYear(title) // Get release year from title

  // Create JSON string for movieId and releaseYear
  val json = s"""{"movieId":$movieId,"releaseYear":$releaseYear}"""
  json
})

// Step 3: Optionally, show the resulting JSONs
movieJSONRDD.take(10).foreach(println)

{"movieId":1,"releaseYear":1995}
{"movieId":2,"releaseYear":1995}
{"movieId":3,"releaseYear":1995}
{"movieId":4,"releaseYear":1995}
{"movieId":5,"releaseYear":1995}
{"movieId":6,"releaseYear":1995}
{"movieId":7,"releaseYear":1995}
{"movieId":8,"releaseYear":1995}
{"movieId":9,"releaseYear":1995}
{"movieId":10,"releaseYear":1995}


extractReleaseYear = > Int = $Lambda$5955/0x0000000802103840@446c45cf
movieJSONRDD = MapPartitionsRDD[38] at map at <console>:40


MapPartitionsRDD[38] at map at <console>:40

In [23]:
movieJSONRDD.saveAsTextFile("hdfs:///user/caseStudy3/json_data")

In [24]:
// Step 3: Read the JSON back into a DataFrame
val jsonDF = spark.read.json("hdfs:///user/caseStudy3/json_data")

// Step 4: Perform the join between moviesDF and jsonDF using movieId to add the releaseYear
val joinedDF = moviesDF
  .join(jsonDF, Seq("movieId"), "left")  // Perform left join on movieId

// Step 5: Show the resulting DataFrame
joinedDF.show()

+-------+--------------------+--------------------+-----------+
|movieId|               title|              genres|releaseYear|
+-------+--------------------+--------------------+-----------+
|      1|    Toy Story (1995)|Adventure|Animati...|       1995|
|      2|      Jumanji (1995)|Adventure|Childre...|       1995|
|      3|Grumpier Old Men ...|      Comedy|Romance|       1995|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|       1995|
|      5|Father of the Bri...|              Comedy|       1995|
|      6|         Heat (1995)|Action|Crime|Thri...|       1995|
|      7|      Sabrina (1995)|      Comedy|Romance|       1995|
|      8| Tom and Huck (1995)|  Adventure|Children|       1995|
|      9| Sudden Death (1995)|              Action|       1995|
|     10|    GoldenEye (1995)|Action|Adventure|...|       1995|
|     11|American Presiden...|Comedy|Drama|Romance|       1995|
|     12|Dracula: Dead and...|       Comedy|Horror|       1995|
|     13|        Balto (1995)|Adventure|

jsonDF = [movieId: bigint, releaseYear: bigint]
joinedDF = [movieId: int, title: string ... 2 more fields]


[movieId: int, title: string ... 2 more fields]

In [25]:
// Step 6: Check for rows where the releaseYear is null (missing)
val missingReleaseYearDF = joinedDF.filter(col("releaseYear").isNull)

// Step 7: Show the rows with missing releaseYear
missingReleaseYearDF.show()

+-------+-----+------+-----------+
|movieId|title|genres|releaseYear|
+-------+-----+------+-----------+
+-------+-----+------+-----------+



missingReleaseYearDF = [movieId: int, title: string ... 2 more fields]


[movieId: int, title: string ... 2 more fields]

In [26]:
spark.stop()